In [196]:
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
from shapely.ops import MultiPoint, Polygon, Point, voronoi_diagram
from pprint import pprint
from scipy.spatial import Voronoi,ConvexHull,voronoi_plot_2d

In [197]:
SUMATIVE_COLOR_BRIGHTNESS = 300
SELECTION_COUNT = 20


BUFFER_CODE = "buffer"
TRIANGLE_CODE = "triangle"

In [198]:
#Reshapes data from the form provided by Lee filters for easier manipulation
def reshapeData(data):
    for name, values in data.items():
        temp = np.array(values[1:])
        data[name] = np.hsplit(temp,2)
        data[name][1]=data[name][1]/100.0
    return data

In [199]:
#Calculates the amount of light absorbed 
#each wavelingth when pasing through filter twice
def getDoubleFilterAbsorb(data):
    data[1]=data[1]**2
    return data

In [200]:
#Get spectra sensitivity from files
#Spectra for IMX 256 extracted form 
#https://s1-dl.theimagingsource.com/api/2.5/packages/publications/whitepapers-cameras/wpspectsens/a5cb0f39-fea8-56ae-8e04-ec20ab4e72c9/wpspectsens_1.30.en_US.pdf
#using https://automeris.io/WebPlotDigitizer/
def initSpectraSensitivity():
    camera = {}
    
    camera['red'] = np.genfromtxt('Red.csv', delimiter=';')
    camera['red'] = np.hsplit(camera['red'],2)
    camera['green'] = np.genfromtxt('Green.csv', delimiter=';')
    camera['green'] = np.hsplit(camera['green'],2)
    camera['blue'] = np.genfromtxt('Blue.csv', delimiter=';')
    camera['blue'] = np.hsplit(camera['blue'],2)
    return camera

In [201]:
def theoreticalRGBValues(filter,camera):
    result = {}
    double_filter = getDoubleFilterAbsorb(filter)
    for key,values in camera.items():
        camera_reaction = double_filter[1] * camera[key][1]
        result[key] = np.sum(camera_reaction)/np.sum(camera[key][1])
        result[key] *= 255
    return result

In [202]:
filter_data = {}
with open("chart-pdf-data.txt") as data:
    filter_data = eval(data.read())
filter_data = reshapeData(filter_data)
theory_rgb = {}
camera_spectra = initSpectraSensitivity()
for key,values in filter_data.items():
    theory_rgb[key] = theoreticalRGBValues(values,camera_spectra)
theory_rgb['327']

{'red': 0.05518738229027276,
 'green': 0.9330367253473104,
 'blue': 0.5460805102576916}

In [203]:

bright_theory = dict(filter((lambda item : (item[1]['red'] + item[1]['green'] + item[1]['blue']>SUMATIVE_COLOR_BRIGHTNESS)),theory_rgb.items()))
print("all colour count: ",len(theory_rgb))
print("filtered colour count:", len(bright_theory))

all colour count:  334
filtered colour count: 106


In [204]:
#Print data in a ternary scatter plot
def showTriplot(data):
    df = pd.DataFrame.from_dict(data,orient="index")
    #print(df)
    color_list = {key: f"rgb({value['red']},{value['green']},{value['blue']})" for (key,value) in data.items()}
    fig = px.scatter_ternary(df,a="red",b="green",c="blue",color=df.index,color_discrete_map=color_list)
    fig.show()

In [205]:
print("Theoretical colors of filters \n\n")
print("Full filter set")
showTriplot(theory_rgb)
print("Bright colored filters only")
showTriplot(bright_theory)



Theoretical colors of filters 


Full filter set


Bright colored filters only


In [206]:
#From given 0-255 rgb values for filters computes euclidian coordinates in terenary diagram
def euclidianCords(rgbData:dict):
    npcolors =np.array([ [val for (k,val) in value.items()] for (key,value) in rgbData.items()])
    npkeys = np.array(list(rgbData.keys()))
    sums = np.sum(npcolors,1)
    normalized_rgb = npcolors/sums.reshape(sums.size,1)
    #print(npkeys.size)
    result={}
    for i in range(0,npkeys.size):
        #print(normalized_rgb[i])
        red = normalized_rgb[i][0]
        green = normalized_rgb[i][1]
        x = 1 - ((red + 2*green)/math.sqrt(3))
        y = red
        result[npkeys[i]] = [x,y]
    return result

In [207]:
def getTriangle():
    return Polygon([[0,0],[0.5,math.sqrt(3)/2],[1,0]])

In [208]:
def voronoi_volumes(points,maskCode,buffer):
    pointSet = MultiPoint(points)
    v = voronoi_diagram(pointSet)
    mask = getTriangle()
    if maskCode == BUFFER_CODE:
        mask = pointSet.envelope.buffer(buffer)
    vol = np.zeros(len(points))
    for polygon in v.geoms:
        polygon = polygon.intersection(mask)
        for i,point in enumerate(points):
            if polygon.intersects(Point(point)):
                vol[i] = polygon.area
    return vol


In [209]:

def selectSpread(data,mask = TRIANGLE_CODE,buffer=0):
    cords = euclidianCords(data)
    selection = dict(data)
    while len(cords) > SELECTION_COUNT : 
        vol = voronoi_volumes(list(cords.values()),mask,buffer)
        vol = dict(zip(cords.keys(),vol))
        minimal = min(vol,key=vol.get)
        del cords[minimal]
        del selection[minimal]
    return selection

In [210]:
def showSelection(data, mask = TRIANGLE_CODE, buffer = 0):
    selection = selectSpread(data, mask, buffer)
    print(f"Selection of filters using {mask} mask and {buffer} padding")
    showTriplot(selection)
    print("Labels of selected filters")
    print(selection.keys())


In [211]:
showSelection(bright_theory)
print("\n")
showSelection(bright_theory, mask=BUFFER_CODE)
print("\n")
showSelection(bright_theory, mask=BUFFER_CODE,buffer = 0.01)
print("\n")
showSelection(bright_theory, mask=BUFFER_CODE,buffer = 0.02)


Selection of filters using triangle mask and 0 padding


Labels of selected filters
dict_keys(['002', '009', '036', '107', '117', '138', '169', '176', '213', '513', '704', '730', 'SC30', '809', 'B71', 'C45', 'LD203', 'M63', 'M91', 'V98'])


Selection of filters using buffer mask and 0 padding


Labels of selected filters
dict_keys(['007', '117', '130', '138', '162', '169', '203', '213', '245', '702', '704', '730', 'SC30', 'B71', 'C45', 'LD203', 'M63', 'M91', 'V98', 'Y02'])


Selection of filters using buffer mask and 0.01 padding


Labels of selected filters
dict_keys(['004', '007', '117', '138', '169', '213', '704', '730', '794', 'SC30', '809', '810', '749', 'B71', 'C45', 'LD203', 'M63', 'M91', 'V98', 'Y02'])


Selection of filters using buffer mask and 0.02 padding


Labels of selected filters
dict_keys(['002', '013', '036', '117', '138', '169', '176', '213', '513', '704', '730', 'SC30', '809', '749', 'B71', 'C45', 'LD203', 'M63', 'M91', 'V98'])
